# POC: Busca Vetorial para Enriquecimento de Endereços DNE

Este notebook implementa e valida a busca vetorial multi-campo com pesos dinâmicos para resolver o problema de scores irreais quando campos estão vazios ou imprecisos.

## Objetivos
1. Construir índices FAISS por campo (logradouro, bairro, cidade)
2. Implementar busca com scoring dinâmico
3. Validar precisão em cenários com CEP errado, abreviações, typos e campos vazios
4. Comparar abordagem multi-campo vs monolítica

In [5]:
import sys
sys.path.append('..')

import pandas as pd
import json
from pathlib import Path
from src.embedding_service import EmbeddingService
from src.index_builder import IndexBuilder
from src.search_engine import SearchEngine

## 1. Carregar Dados

In [6]:
# Carregar dataset DNE
data_path = Path('../data')
df_dne = pd.read_parquet(data_path / 'dne_sample.parquet')
df_queries = pd.read_parquet(data_path / 'test_queries.parquet')

print(f"Dataset DNE: {len(df_dne)} registros")
print(f"Queries de teste: {len(df_queries)} queries")
print(f"\nDistribuição de queries por categoria:")
print(df_queries['category'].value_counts())

Dataset DNE: 10000 registros
Queries de teste: 150 queries

Distribuição de queries por categoria:
category
abbreviation    60
cep_wrong       45
typo            30
empty_fields    15
Name: count, dtype: int64


## 2. Inicializar Serviços e Construir Índices

In [7]:
# Inicializa serviço de embeddings com modelo neuralmind
embedding_service = EmbeddingService(model_name="neuralmind/bert-base-portuguese-cased")

No sentence-transformers model found with name neuralmind/bert-base-portuguese-cased. Creating a new one with mean pooling.


Carregando modelo de embeddings: neuralmind/bert-base-portuguese-cased


config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

c:\Users\rik_d\OneDrive\Documentos\Projetos\Backend\IA\genia\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rik_d\.cache\huggingface\hub\models--neuralmind--bert-base-portuguese-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Exception ignored in: <function tqdm.__del__ at 0x0000018236052340>
Traceback (most recent call last):
  File "c:\Users\rik_d\OneDrive\Documentos\Projetos\Backend\IA\genia\venv\Lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "c:\Users\rik_d\OneDrive\Documentos\Projetos\Backend\IA\genia\venv\Lib\site-packages\tqdm\notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
    ^^^^^^^^^
AttributeError: 'tqdm' object has no attribute 'disp'


In [8]:
# Constrói índices FAISS
index_builder = IndexBuilder(embedding_service)
indices = index_builder.build_indices(df_dne)

Construindo índices FAISS para 10000 endereços


Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

In [9]:
# Salva índices para reutilização
indices_path = data_path / 'indices'
index_builder.save_indices(str(indices_path))
print(f"\nÍndices salvos em: {indices_path}")


Índices salvos em: ..\data\indices


## 3. Inicializar Motor de Busca

In [10]:
# Inicializa motor de busca
search_engine = SearchEngine(
    embedding_service=embedding_service,
    indices=indices,
    dataframe=df_dne
)

print("Motor de busca inicializado")
print(f"Threshold de confiança alta: {search_engine.confidence_threshold}")

Motor de busca inicializado
Threshold de confiança alta: 0.8


## 4. Testes Exploratórios

In [11]:
# Teste 1: Query limpa (baseline)
sample = df_dne.iloc[100]
query = {
    'logradouro': sample['logradouro'],
    'bairro': sample['bairro'],
    'cidade': sample['cidade'],
    'uf': sample['uf'],
    'cep': sample['cep']
}

print("=== Teste 1: Query Limpa (Baseline) ===")
print(f"Query: {query}\n")
result = search_engine.search(query, top_k=3)
print(json.dumps(json.loads(result), indent=2, ensure_ascii=False))

=== Teste 1: Query Limpa (Baseline) ===
Query: {'logradouro': 'Estrada Paraná', 'bairro': 'Vila Industrial', 'cidade': 'Itabuna', 'uf': 'BA', 'cep': '40224-644'}

{
  "results": [
    {
      "address": {
        "logradouro": "Estrada Paraná",
        "bairro": "Vila Industrial",
        "cidade": "Itabuna",
        "uf": "BA",
        "cep": "40224-644"
      },
      "score": 1.0,
      "confidence": "high",
      "field_scores": {
        "logradouro": 1.0,
        "bairro": 1.0,
        "cidade": 1.0,
        "cep": 1.0
      }
    },
    {
      "address": {
        "logradouro": "Estrada Paraná",
        "bairro": "Parque Industrial",
        "cidade": "Montes Claros",
        "uf": "MG",
        "cep": "30224-024"
      },
      "score": 0.4000000059604645,
      "confidence": "low",
      "field_scores": {
        "logradouro": 1.0,
        "cep": 0.0
      }
    },
    {
      "address": {
        "logradouro": "Estrada Paraná",
        "bairro": "Parque Oeste",
        "cida

In [12]:
# Teste 2: CEP errado mas rua/bairro corretos
query_cep_wrong = query.copy()
query_cep_wrong['cep'] = '20000-000'  # CEP do RJ (se original não for RJ)

print("\n=== Teste 2: CEP Errado ===")
print(f"Query: {query_cep_wrong}\n")
result = search_engine.search(query_cep_wrong, top_k=3)
print(json.dumps(json.loads(result), indent=2, ensure_ascii=False))


=== Teste 2: CEP Errado ===
Query: {'logradouro': 'Estrada Paraná', 'bairro': 'Vila Industrial', 'cidade': 'Itabuna', 'uf': 'BA', 'cep': '20000-000'}

{
  "results": [
    {
      "address": {
        "logradouro": "Estrada Paraná",
        "bairro": "Vila Industrial",
        "cidade": "Itabuna",
        "uf": "BA",
        "cep": "40224-644"
      },
      "score": 0.7000000476837158,
      "confidence": "medium",
      "field_scores": {
        "logradouro": 1.0,
        "bairro": 1.0,
        "cidade": 1.0,
        "cep": 0.0
      }
    },
    {
      "address": {
        "logradouro": "Estrada Paraná",
        "bairro": "Parque Industrial",
        "cidade": "Montes Claros",
        "uf": "MG",
        "cep": "30224-024"
      },
      "score": 0.4000000059604645,
      "confidence": "low",
      "field_scores": {
        "logradouro": 1.0,
        "cep": 0.0
      }
    },
    {
      "address": {
        "logradouro": "Estrada Paraná",
        "bairro": "Parque Oeste",
       

In [13]:
# Teste 3: Apenas bairro e cidade (campos vazios)
query_partial = {
    'logradouro': '',
    'bairro': sample['bairro'],
    'cidade': sample['cidade'],
    'uf': sample['uf'],
    'cep': ''
}

print("\n=== Teste 3: Campos Vazios (só bairro + cidade) ===")
print(f"Query: {query_partial}\n")
result = search_engine.search(query_partial, top_k=5)
print(json.dumps(json.loads(result), indent=2, ensure_ascii=False))


=== Teste 3: Campos Vazios (só bairro + cidade) ===
Query: {'logradouro': '', 'bairro': 'Vila Industrial', 'cidade': 'Itabuna', 'uf': 'BA', 'cep': ''}

{
  "results": [
    {
      "address": {
        "logradouro": "Estrada Paraná",
        "bairro": "Vila Industrial",
        "cidade": "Itabuna",
        "uf": "BA",
        "cep": "40224-644"
      },
      "score": 1.0,
      "confidence": "high",
      "field_scores": {
        "bairro": 1.0,
        "cidade": 1.0
      }
    },
    {
      "address": {
        "logradouro": "Rodovia do Sol",
        "bairro": "Vila Industrial",
        "cidade": "Itabuna",
        "uf": "BA",
        "cep": "40812-062"
      },
      "score": 1.0,
      "confidence": "high",
      "field_scores": {
        "bairro": 1.0,
        "cidade": 1.0
      }
    },
    {
      "address": {
        "logradouro": "Rodovia ePdro Álvares Cabral",
        "bairro": "Vila Industrial",
        "cidade": "Itabuna",
        "uf": "BA",
        "cep": "40652-773"


In [14]:
# Teste 4: Com abreviações
query_abbr = query.copy()
if query_abbr['logradouro'].startswith('Rua'):
    query_abbr['logradouro'] = query_abbr['logradouro'].replace('Rua', 'R.', 1)
elif query_abbr['logradouro'].startswith('Avenida'):
    query_abbr['logradouro'] = query_abbr['logradouro'].replace('Avenida', 'Av.', 1)

print("\n=== Teste 4: Abreviações ===")
print(f"Query: {query_abbr}\n")
result = search_engine.search(query_abbr, top_k=3)
print(json.dumps(json.loads(result), indent=2, ensure_ascii=False))


=== Teste 4: Abreviações ===
Query: {'logradouro': 'Estrada Paraná', 'bairro': 'Vila Industrial', 'cidade': 'Itabuna', 'uf': 'BA', 'cep': '40224-644'}

{
  "results": [
    {
      "address": {
        "logradouro": "Estrada Paraná",
        "bairro": "Vila Industrial",
        "cidade": "Itabuna",
        "uf": "BA",
        "cep": "40224-644"
      },
      "score": 1.0,
      "confidence": "high",
      "field_scores": {
        "logradouro": 1.0,
        "bairro": 1.0,
        "cidade": 1.0,
        "cep": 1.0
      }
    },
    {
      "address": {
        "logradouro": "Estrada Paraná",
        "bairro": "Parque Industrial",
        "cidade": "Montes Claros",
        "uf": "MG",
        "cep": "30224-024"
      },
      "score": 0.4000000059604645,
      "confidence": "low",
      "field_scores": {
        "logradouro": 1.0,
        "cep": 0.0
      }
    },
    {
      "address": {
        "logradouro": "Estrada Paraná",
        "bairro": "Parque Oeste",
        "cidade": "Cric

## 5. Validação Sistemática por Categoria

In [15]:
def evaluate_queries(queries_df, search_engine, top_k=5):
    """Avalia precisão das queries"""
    results = []
    
    for idx, row in queries_df.iterrows():
        query = {
            'logradouro': row['logradouro'],
            'bairro': row['bairro'],
            'cidade': row['cidade'],
            'uf': row['uf'],
            'cep': row['cep']
        }
        
        # Busca
        result_json = search_engine.search(query, top_k=top_k)
        result = json.loads(result_json)
        
        # Verifica se o endereço esperado está nos resultados
        expected_idx = row['expected_index']
        expected_address = df_dne.iloc[expected_idx]
        
        found = False
        rank = -1
        best_score = 0.0
        
        for i, res in enumerate(result['results']):
            score = res['score']
            if score > best_score:
                best_score = score
            
            # Verifica match (comparando campos principais)
            addr = res['address']
            if (addr['logradouro'] == expected_address['logradouro'] and
                addr['cidade'] == expected_address['cidade'] and
                addr['cep'] == expected_address['cep']):
                found = True
                rank = i + 1
                break
        
        results.append({
            'category': row['category'],
            'found': found,
            'rank': rank,
            'best_score': best_score,
            'has_high_confidence': best_score >= search_engine.confidence_threshold
        })
    
    return pd.DataFrame(results)

In [16]:
# Avalia todas as queries
print("Avaliando queries de teste...")
evaluation_results = evaluate_queries(df_queries, search_engine, top_k=5)

Avaliando queries de teste...


In [17]:
# Métricas gerais
print("\n=== Métricas Gerais ===")
print(f"Precisão@5: {evaluation_results['found'].mean():.2%}")
print(f"Queries com alta confiança (score ≥ 0.8): {evaluation_results['has_high_confidence'].mean():.2%}")
print(f"Score médio (melhor resultado): {evaluation_results['best_score'].mean():.3f}")
print(f"\nDistribuição de ranks (quando encontrado):")
print(evaluation_results[evaluation_results['found']]['rank'].value_counts().sort_index())


=== Métricas Gerais ===
Precisão@5: 98.00%
Queries com alta confiança (score ≥ 0.8): 56.67%
Score médio (melhor resultado): 0.833

Distribuição de ranks (quando encontrado):
rank
1    146
2      1
Name: count, dtype: int64


In [18]:
# Métricas por categoria
print("\n=== Métricas por Categoria ===")
for category in evaluation_results['category'].unique():
    cat_data = evaluation_results[evaluation_results['category'] == category]
    print(f"\n{category.upper()}:")
    print(f"  Precisão@5: {cat_data['found'].mean():.2%}")
    print(f"  Alta confiança: {cat_data['has_high_confidence'].mean():.2%}")
    print(f"  Score médio: {cat_data['best_score'].mean():.3f}")


=== Métricas por Categoria ===

TYPO:
  Precisão@5: 100.00%
  Alta confiança: 40.00%
  Score médio: 0.731

ABBREVIATION:
  Precisão@5: 100.00%
  Alta confiança: 100.00%
  Score médio: 0.969

CEP_WRONG:
  Precisão@5: 97.78%
  Alta confiança: 0.00%
  Score médio: 0.682

EMPTY_FIELDS:
  Precisão@5: 86.67%
  Alta confiança: 86.67%
  Score médio: 0.951


## 6. Análise de Casos Específicos

In [19]:
# Exemplos de cada categoria
print("\n=== Exemplos Representativos ===")

for category in ['cep_wrong', 'abbreviation', 'typo', 'empty_fields']:
    print(f"\n{'='*60}")
    print(f"CATEGORIA: {category.upper()}")
    print('='*60)
    
    # Pega 2 exemplos da categoria
    cat_queries = df_queries[df_queries['category'] == category].head(2)
    
    for _, row in cat_queries.iterrows():
        query = {
            'logradouro': row['logradouro'],
            'bairro': row['bairro'],
            'cidade': row['cidade'],
            'uf': row['uf'],
            'cep': row['cep']
        }
        
        expected = df_dne.iloc[row['expected_index']]
        
        print(f"\nQuery: {query}")
        print(f"Esperado: {expected.to_dict()}")
        
        result_json = search_engine.search(query, top_k=3)
        result = json.loads(result_json)
        
        print(f"\nTop-3 Resultados:")
        for i, res in enumerate(result['results'], 1):
            print(f"  #{i} [score={res['score']:.3f}, conf={res['confidence']}]:")
            print(f"      {res['address']}")
        print()
    
    print()


=== Exemplos Representativos ===

CATEGORIA: CEP_WRONG

Query: {'logradouro': 'Largo Dom Pedro II', 'bairro': 'Parque Europa', 'cidade': 'Aparecida de Goiânia', 'uf': 'GO', 'cep': '88490-104'}
Esperado: {'logradouro': 'Largo Dom Pedro II', 'bairro': 'Parque Europa', 'cidade': 'Aparecida de Goiânia', 'uf': 'GO', 'cep': '73537-072'}

Top-3 Resultados:
  #1 [score=0.700, conf=medium]:
      {'logradouro': 'Largo Dom Pedro II', 'bairro': 'Parque Europa', 'cidade': 'Aparecida de Goiânia', 'uf': 'GO', 'cep': '73537-072'}
  #2 [score=0.400, conf=low]:
      {'logradouro': 'Largo Dom Pedro II', 'bairro': 'Cidade Serra', 'cidade': 'Uberlândia', 'uf': 'MG', 'cep': '30733-185'}
  #3 [score=0.400, conf=low]:
      {'logradouro': 'Largo Dom Pedro II', 'bairro': 'Cidade São Francisco', 'cidade': 'Ribeirão Preto', 'uf': 'SP', 'cep': '01692-506'}


Query: {'logradouro': 'Estrada Presidente Vargas', 'bairro': 'Parque do Sol', 'cidade': 'Foz do Iguaçu', 'uf': 'PR', 'cep': '40470-693'}
Esperado: {'logra

## 7. Conclusões e Próximos Passos

In [20]:
print("\n" + "="*70)
print("RESUMO DA POC: Busca Vetorial Multi-Campo para Enriquecimento de DNE")
print("="*70)

print("\n✅ IMPLEMENTADO:")
print("  - Embeddings multi-campo com rufimelo/bert-base-portuguese-cased-nli-assin-2")
print("  - Índices FAISS separados por campo (logradouro, bairro, cidade)")
print("  - Scoring dinâmico com pesos ajustados por query")
print("  - Normalização de texto (unidecode, abreviações, lowercase)")
print("  - Threshold de confiança 0.8 para sugestões de alta qualidade")
print("  - Retorno em JSON estruturado com scores por campo")

print("\n📊 RESULTADOS:")
print(f"  - Precisão@5 geral: {evaluation_results['found'].mean():.2%}")
print(f"  - Queries com alta confiança: {evaluation_results['has_high_confidence'].mean():.2%}")
print(f"  - Score médio: {evaluation_results['best_score'].mean():.3f}")

print("\n💡 VANTAGENS DA ABORDAGEM MULTI-CAMPO:")
print("  - Campos vazios não poluem o score geral")
print("  - CEP errado não elimina match de rua correta")
print("  - Sistema pode sugerir CEP correto quando outros campos têm score ≥ 0.8")
print("  - Pesos dinâmicos se adaptam aos campos disponíveis na query")

print("\n🎯 PRÓXIMOS PASSOS (se necessário):")
print("  1. Ajustar pesos baseado nos resultados de validação")
print("  2. Testar com dados reais do DNE (se disponível)")
print("  3. Implementar cache de embeddings para queries frequentes")
print("  4. Adicionar filtros geográficos (estado/cidade) para reduzir espaço de busca")
print("  5. Fine-tuning do modelo em dataset de endereços brasileiros")

print("\n" + "="*70)


RESUMO DA POC: Busca Vetorial Multi-Campo para Enriquecimento de DNE

✅ IMPLEMENTADO:
  - Embeddings multi-campo com rufimelo/bert-base-portuguese-cased-nli-assin-2
  - Índices FAISS separados por campo (logradouro, bairro, cidade)
  - Scoring dinâmico com pesos ajustados por query
  - Normalização de texto (unidecode, abreviações, lowercase)
  - Threshold de confiança 0.8 para sugestões de alta qualidade
  - Retorno em JSON estruturado com scores por campo

📊 RESULTADOS:
  - Precisão@5 geral: 98.00%
  - Queries com alta confiança: 56.67%
  - Score médio: 0.833

💡 VANTAGENS DA ABORDAGEM MULTI-CAMPO:
  - Campos vazios não poluem o score geral
  - CEP errado não elimina match de rua correta
  - Sistema pode sugerir CEP correto quando outros campos têm score ≥ 0.8
  - Pesos dinâmicos se adaptam aos campos disponíveis na query

🎯 PRÓXIMOS PASSOS (se necessário):
  1. Ajustar pesos baseado nos resultados de validação
  2. Testar com dados reais do DNE (se disponível)
  3. Implementar cache